## Pretrained ResNet18 (Fixed)

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from tqdm import tqdm
from torchvision.models import resnet18
import numpy as np

In [2]:
args = {
    'learning_rate': 5e-2,
    'batch_size': 16,
    'num_worker': 4,
    'random_seed': 8771795,
    'augmentation': False,
    'num_epoch': 20,
    'cuda': '3'
}

In [3]:
# set device
os.environ["CUDA_VISIBLE_DEVICES"] = args['cuda']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set random seed
torch.random.manual_seed(args['random_seed'])

# Define transformation
test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_valid_transform = test_transform
if args['augmentation']:
    train_valid_transform = transforms.Compose([
    #         transforms.RandomResizedCrop((224, 224)),
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.RandomErasing(),
            transforms.Normalize((0.5,), (0.5,))
        ])

# Load dataset
require_download = os.path.exists('./dataset')
train_valid_dataset = torchvision.datasets.FashionMNIST('./dataset', train=True, transform=train_valid_transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST('./dataset', train=False, transform=test_transform, download=True)

# Split train and validation
torch.random.manual_seed(args['random_seed'])
train_dataset, valid_dataset = torch.utils.data.random_split(train_valid_dataset, [54000, 6000])

# Generate dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=args['num_worker'])
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])

In [4]:
model = resnet18(pretrained=True)

In [5]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [6]:
model.fc = Identity()

In [7]:
model.eval()
print('total parameters:', sum(p.numel() for p in model.parameters()))

total parameters: 11176512


In [8]:
if not os.path.exists('./features'):
    os.mkdir('./features')

In [9]:
model = model.to(device)

In [10]:
train_feats, valid_feats, test_feats = [], [], []
train_labels, valid_labels, test_labels = [], [], []
for iteration, (x, y) in tqdm(enumerate(train_loader)):
    x, y = x.to(device), y.to(device)
    features = model(x)
    train_feats.append(features.cpu().detach().numpy())
    train_labels.append(y.cpu().detach().numpy())
    del x, y, features

3375it [00:43, 76.95it/s]


In [11]:
train_feats = np.concatenate(train_feats)

In [12]:
train_feats.shape

(54000, 512)

In [13]:
train_labels = np.concatenate(train_labels)
train_labels.shape

(54000,)

In [14]:
np.save('features/resnet18_train_feat.npy', train_feats)
np.save('features/resnet18_train_label.npy', train_labels)

In [15]:
del train_labels, train_feats

In [16]:
for iteration, (x, y) in tqdm(enumerate(valid_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    features = model(x)
    valid_feats.append(features.cpu().detach().numpy())
    valid_labels.append(y.cpu().detach().numpy())
    del x, y, features

375it [00:04, 76.21it/s]


In [17]:
valid_feats = np.concatenate(valid_feats)
valid_feats.shape

(6000, 512)

In [18]:
valid_labels = np.concatenate(valid_labels)
valid_labels.shape

(6000,)

In [19]:
np.save('features/resnet18_valid_feat.npy', valid_feats)
np.save('features/resnet18_valid_label.npy', valid_labels)

In [20]:
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    features = model(x)
    test_feats.append(features.cpu().detach().numpy())
    test_labels.append(y.cpu().detach().numpy())
    del x, y, features

625it [00:08, 75.55it/s]


In [21]:
test_feats = np.concatenate(test_feats)
test_feats.shape

(10000, 512)

In [22]:
test_labels = np.concatenate(test_labels)
test_labels.shape

(10000,)

In [23]:
np.save('features/resnet18_test_feat.npy', test_feats)
np.save('features/resnet18_test_label.npy', test_labels)

## Pretrained ResNet18 (Finetune)

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from tqdm import tqdm
from torchvision.models import resnet18
import numpy as np

In [2]:
args = {
    'learning_rate': 1e-3,
    'batch_size': 64,
    'num_worker': 16,
    'random_seed': 8771795,
    'augmentation': False,
    'num_epoch': 10,
    'cuda': '3'
}

In [3]:
# set device
os.environ["CUDA_VISIBLE_DEVICES"] = args['cuda']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set random seed
torch.random.manual_seed(args['random_seed'])

# Define transformation
test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_valid_transform = test_transform
if args['augmentation']:
    train_valid_transform = transforms.Compose([
    #         transforms.RandomResizedCrop((224, 224)),
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.RandomErasing(),
            transforms.Normalize((0.5,), (0.5,))
        ])

# Load dataset
require_download = os.path.exists('./dataset')
train_valid_dataset = torchvision.datasets.FashionMNIST('./dataset', train=True, transform=train_valid_transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST('./dataset', train=False, transform=test_transform, download=True)

# Split train and validation
torch.random.manual_seed(args['random_seed'])
train_dataset, valid_dataset = torch.utils.data.random_split(train_valid_dataset, [54000, 6000])

# Generate dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=args['num_worker'])
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])

### Train only the last FC layer

In [8]:
model = resnet18(pretrained=True)
for param in model.parameters():
    param.require_grad = False
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [9]:
model.fc = nn.Sequential(nn.Linear(in_features=512, out_features=64, bias=True),
                        nn.ReLU(),
                        nn.Linear(in_features=64, out_features=10, bias=True))

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

In [11]:
model.to(device)
for epoch in range(20):
    total_train = 0
    for iteration, (x, y) in tqdm(enumerate(train_loader)):
        model.train()
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = criterion(pred, y)
        total_train += loss.item()
        loss.backward()
        optimizer.step()
        del x, y, pred
    print('epoch {}: training loss: {}'.format(epoch+1, total_train/(iteration+1)))
    torch.cuda.empty_cache()

    val_acc = 0
    for iteration, (x, y) in tqdm(enumerate(valid_loader)):
        model.eval()
        x, y = x.to(device), y.to(device)
        out = model(x)
        pred = torch.argmax(out, dim=1)
        acc = sum(pred == y)
        val_acc += acc.item()
    print('epoch {}: valid acc: {}'.format(epoch+1, val_acc/6000))

844it [01:40,  8.40it/s]


epoch 1: training loss: 0.8268528820425978


94it [00:04, 20.90it/s]

epoch 1: valid acc: 0.7083333333333334



844it [01:39,  8.47it/s]


...

epoch 19: valid acc: 0.9253333333333333



844it [01:40,  8.36it/s]


epoch 20: training loss: 0.17510400716462549


94it [00:04, 20.80it/s]

epoch 20: valid acc: 0.9265


In [8]:
test_acc = 0
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    out = model(x)
    pred = torch.argmax(out, dim=1)
    acc = sum(pred == y)
    test_acc += acc.item()
print('test acc: {}'.format(test_acc/10000))

313it [00:07, 43.65it/s]

test acc: 0.8273


In [12]:
test_acc = 0
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    out = model(x)
    pred = torch.argmax(out, dim=1)
    acc = sum(pred == y)
    test_acc += acc.item()
print('test acc: {}'.format(test_acc/10000))

157it [00:08, 18.04it/s]

test acc: 0.9214


In [13]:
torch.save(model.state_dict(), 'resnet18_before_2layers.pt')

### Fine-tune all the layers

In [54]:
model = resnet18()
model.fc = nn.Sequential(nn.Linear(in_features=512, out_features=64, bias=True),
                        nn.ReLU(),
                        nn.Linear(in_features=64, out_features=10, bias=True))# nn.Linear(512, 10)
model.load_state_dict(torch.load('resnet18_before_2layers.pt'))

<All keys matched successfully>

In [55]:
for param in model.parameters():
    param.require_grad = True

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=2.5e-5)

In [57]:
model.to(device)
for epoch in range(10):
    total_train = 0
    for iteration, (x, y) in tqdm(enumerate(train_loader)):
        model.train()
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = criterion(pred, y)
        total_train += loss.item()
        loss.backward()
        optimizer.step()
        del x, y, pred
    print('epoch {}: training loss: {}'.format(epoch+1, total_train/(iteration+1)))
    torch.cuda.empty_cache()

    val_acc = 0
    for iteration, (x, y) in tqdm(enumerate(valid_loader)):
        model.eval()
        x, y = x.to(device), y.to(device)
        out = model(x)
        pred = torch.argmax(out, dim=1)
        acc = sum(pred == y)
        val_acc += acc.item()
    print('epoch {}: valid acc: {}'.format(epoch+1, val_acc/6000))

844it [01:40,  8.43it/s]


epoch 1: training loss: 0.1609403055571726


94it [00:04, 21.00it/s]

epoch 1: valid acc: 0.9315



844it [01:39,  8.48it/s]


...

epoch 9: valid acc: 0.9343333333333333



844it [01:40,  8.38it/s]


epoch 10: training loss: 0.07937081417254221


94it [00:04, 20.71it/s]

epoch 10: valid acc: 0.936


In [58]:
test_acc = 0
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    out = model(x)
    pred = torch.argmax(out, dim=1)
    acc = sum(pred == y)
    test_acc += acc.item()
print('test acc: {}'.format(test_acc/10000))

157it [00:07, 22.32it/s]

test acc: 0.9322


In [59]:
torch.save(model.state_dict(), 'resnet18_after_2layers.pt')

## Feature Extraction & Compute F1

In [1]:
import os
import torch
import torch.nn as nn
from sklearn.metrics import precision_recall_curve, average_precision_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchvision.models import resnet18
from torchvision import transforms
from tqdm import tqdm

In [2]:
args = {
    'learning_rate': 1e-3,
    'batch_size': 32,
    'num_worker': 4,
    'random_seed': 8771795,
    'augmentation': False,
    'num_epoch': 10,
    'cuda': '5'
}

In [3]:
# set device
os.environ["CUDA_VISIBLE_DEVICES"] = args['cuda']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set random seed
torch.random.manual_seed(args['random_seed'])

# Define transformation
test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_valid_transform = test_transform
if args['augmentation']:
    train_valid_transform = transforms.Compose([
    #         transforms.RandomResizedCrop((224, 224)),
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.RandomErasing(),
            transforms.Normalize((0.5,), (0.5,))
        ])

# Load dataset
require_download = os.path.exists('./dataset')
train_valid_dataset = torchvision.datasets.FashionMNIST('./dataset', train=True, transform=train_valid_transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST('./dataset', train=False, transform=test_transform, download=True)

# Split train and validation
torch.random.manual_seed(args['random_seed'])
train_dataset, valid_dataset = torch.utils.data.random_split(train_valid_dataset, [54000, 6000])

# Generate dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=args['num_worker'])
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])

In [4]:
# pretrained ResNet18
model = resnet18()
# model.fc = nn.Linear(512, 10)
model.fc = nn.Sequential(nn.Linear(in_features=512, out_features=64, bias=True),
                        nn.ReLU(),
                        nn.Linear(in_features=64, out_features=10, bias=True))
model.load_state_dict(torch.load('resnet18_before_2layers.pt'))

<All keys matched successfully>

In [5]:
model.to(device)
test_acc = 0
labels, preds = [], []
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    out = model(x)
    pred = torch.argmax(out, dim=1)
    acc = sum(pred == y)
    test_acc += acc.item()
    labels.append(y.cpu().detach().numpy())
    preds.append(pred.cpu().detach().numpy())
    del out, pred, x, y
print('test acc: {}'.format(test_acc/10000))

313it [00:08, 37.25it/s]

test acc: 0.9214


In [6]:
labels = np.concatenate(labels)
preds = np.concatenate(preds)
assert labels.shape == preds.shape

In [10]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import recall_score, f1_score, precision_score, accuracy_score

In [11]:
from tabulate import tabulate

In [12]:
precision, recall, fscore, support = score(labels, preds)
pr = precision_score(labels, preds, average='macro')
re = recall_score(labels, preds, average='macro')
fs = f1_score(labels, preds, average='macro')
ac = accuracy_score(labels, preds)

table = [[i, p, r, f, s] for i, p, r, f, s in zip(range(10), precision, recall, fscore, support)]
table.append(['overall', pr, re, fs, 10000])

In [13]:
print(tabulate(table, headers=['class', 'precision', 'recall', 'f1', 'support']))

class      precision    recall        f1    support
-------  -----------  --------  --------  ---------
0           0.895127    0.845   0.869342       1000
1           0.993896    0.977   0.985376       1000
2           0.917184    0.886   0.901322       1000
3           0.905314    0.937   0.920885       1000
4           0.882773    0.866   0.874306       1000
5           0.988832    0.974   0.98136        1000
6           0.741636    0.798   0.768786       1000
7           0.95155     0.982   0.966535       1000
8           0.973373    0.987   0.980139       1000
9           0.977642    0.962   0.969758       1000
overall     0.922733    0.9214  0.921781      10000


In [14]:
del model

In [15]:
# pretrained ResNet18 + finetune
model = resnet18()
model.fc = nn.Sequential(nn.Linear(in_features=512, out_features=64, bias=True),
                        nn.ReLU(),
                        nn.Linear(in_features=64, out_features=10, bias=True))
model.load_state_dict(torch.load('resnet18_after_2layers.pt'))

<All keys matched successfully>

In [16]:
model.to(device)
test_acc = 0
labels, preds = [], []
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    out = model(x)
    pred = torch.argmax(out, dim=1)
    acc = sum(pred == y)
    test_acc += acc.item()
    labels.append(y.cpu().detach().numpy())
    preds.append(pred.cpu().detach().numpy())
    del out, pred, x, y
print('test acc: {}'.format(test_acc/10000))

313it [00:07, 41.34it/s]

test acc: 0.9322


In [17]:
labels = np.concatenate(labels)
preds = np.concatenate(preds)
assert labels.shape == preds.shape

In [18]:
precision, recall, fscore, support = score(labels, preds)
pr = precision_score(labels, preds, average='macro')
re = recall_score(labels, preds, average='macro')
fs = f1_score(labels, preds, average='macro')
ac = accuracy_score(labels, preds)

table = [[i, p, r, f, s] for i, p, r, f, s in zip(range(10), precision, recall, fscore, support)]
table.append(['overall', pr, re, fs, 10000])
print(tabulate(table, headers=['class', 'precision', 'recall', 'f1', 'support']))

class      precision    recall        f1    support
-------  -----------  --------  --------  ---------
0           0.895277    0.872   0.883485       1000
1           0.993939    0.984   0.988945       1000
2           0.906746    0.914   0.910359       1000
3           0.939271    0.928   0.933602       1000
4           0.875954    0.918   0.896484       1000
5           0.988922    0.982   0.985449       1000
6           0.806288    0.795   0.800604       1000
7           0.955209    0.981   0.967933       1000
8           0.983085    0.988   0.985536       1000
9           0.978593    0.96    0.969207       1000
overall     0.932329    0.9322  0.93216       10000


In [19]:
# save labels & features
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
model.fc = Identity()

In [20]:
train_feats, valid_feats, test_feats = [], [], []
train_labels, valid_labels, test_labels = [], [], []
for iteration, (x, y) in tqdm(enumerate(train_loader)):
    x, y = x.to(device), y.to(device)
    features = model(x)
    train_feats.append(features.cpu().detach().numpy())
    train_labels.append(y.cpu().detach().numpy())
    del x, y, features
train_feats = np.concatenate(train_feats)
train_labels = np.concatenate(train_labels)

np.save('features/resnet18_finetune_2layers_train_feat.npy', train_feats)
np.save('features/resnet18_finetune_2layers_train_label.npy', train_labels)

del train_feats, train_labels

1688it [00:39, 42.52it/s]


In [21]:
for iteration, (x, y) in tqdm(enumerate(valid_loader)):
    x, y = x.to(device), y.to(device)
    features = model(x)
    valid_feats.append(features.cpu().detach().numpy())
    valid_labels.append(y.cpu().detach().numpy())
    del x, y, features
valid_feats = np.concatenate(valid_feats)
valid_labels = np.concatenate(valid_labels)


np.save('features/resnet18_finetune_2layers_valid_feat.npy', valid_feats)
np.save('features/resnet18_finetune_2layers_valid_label.npy', valid_labels)

del valid_feats, valid_labels

188it [00:04, 40.67it/s]


In [22]:
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    x, y = x.to(device), y.to(device)
    features = model(x)
    test_feats.append(features.cpu().detach().numpy())
    test_labels.append(y.cpu().detach().numpy())
    del x, y, features
test_feats = np.concatenate(test_feats)
test_labels = np.concatenate(test_labels)

np.save('features/resnet18_finetune_2layers_test_feat.npy', test_feats)
np.save('features/resnet18_finetune_2layers_test_label.npy', test_labels)

313it [00:08, 37.25it/s]


In [23]:
test_feats.shape 

(10000, 512)